In [ ]:
# default_exp s3funcs

%load_ext autoreload
%autoreload 2

In [ ]:
# export

import boto3
import os
import getpass
from io import StringIO
from configparser import ConfigParser


def check_if_creds_exist():
    user = getpass.getuser()
    if os.path.isfile("/home/{}/.aws/credentials".format(user)):
        pass
    elif os.path.isfile("/Users/{}/.aws/credentials".format(user)):
        pass
    else:
        raise Exception("~/.aws/credentials file not found.")


class S3Bucket(object):
    """
    Simple S3 bucket abstracting away boto3 complexity.
    """

    def __init__(self, bucket_name):
        check_if_creds_exist()
        self.bucket_name = bucket_name
        self.s3 = boto3.resource("s3")
        self.bucket = self.s3.Bucket(self.bucket_name)
        self.config = ConfigParser()
        self.parse_config()

    def parse_config(self):
        user = getpass.getuser()
        if os.path.isfile("/home/{}/.aws/credentials".format(user)):
            self.config.read("/home/{}/.aws/credentials".format(user))
        elif os.path.isfile("/Users/{}/.aws/credentials".format(user)):
            self.config.read("/Users/{}/.aws/credentials".format(user))
        self.aws_access_key_id = self.config['default']['aws_access_key_id']
        self.aws_secret_access_key = self.config['default']['aws_secret_access_key']

    def upload(self, file, s3_loc):
        try:
            self.bucket.upload_file(filename=file, bucket=self.bucket_name, key=s3_loc)
        except:
            print("Unable to upload {0} to {1}".format(file, self.bucket_name))

    def download_obj(self, s3_loc, dataobj):
        """
        Writes the data from object to `dataobj`.
        """
        self.bucket.download_fileobj(s3_loc, dataobj)

    def download(self, s3_loc, output_file):
        self.bucket.download_file(s3_loc, output_file)

    def write_dataframe(self, dataframe, s3_loc):
        csv_buffer = StringIO()
        dataframe.to_csv(csv_buffer, sep = "\t", index = False, header = None, float_format='%.0f')
        self.bucket.put_object(Body = csv_buffer.getvalue(), Key = s3_loc)

    def delete_obj(self):
        pass